In [1]:
%pylab inline
import numpy as np
import scipy
import math
import time
from scipy.sparse.linalg import onenormest, splu, LinearOperator
from scipy.sparse import coo_matrix, block_diag, identity, hstack
import matplotlib.pyplot as plt
from pyiga import assemble, bspline, vform, geometry, vis, solvers, utils, topology
#from patchmesh import *
#from sksparse.cholmod import cholesky
#from patchmesh3D import *
#from multipatch import *

np.set_printoptions(linewidth=100000)
np.set_printoptions(precision=2)
np.set_printoptions(formatter={'float': lambda x: "{0: 0.3f}".format(x) if x>=0 else "{0:0.3f}".format(x)})

Populating the interactive namespace from numpy and matplotlib


In [2]:
def condest(A):
    luA = splu(A)
    iA = LinearOperator(luA.shape, matvec = lambda x : luA.solve(x), rmatvec = lambda x : luA.solve(x))
    return onenormest(iA)*onenormest(A)

def pcg(Afuns, f, tol = 1e-5, maxit = 100, pfuns = 1, output = False):     
    maxit = int(maxit)
    if not callable(pfuns):
        pfun = lambda x : x
        # splu_pfun = sp.linalg.splu(pfuns,permc_spec='COLAMD')
        # pfun = lambda x : splu_pfun.solve(x)
    else:
        pfun = pfuns
    if not callable(Afuns):
        Afun = lambda x : Afuns@x
    else:
        Afun = Afuns
    if not isinstance(f,np.ndarray):
        d = f.A.ravel()
    else:
        d = f.ravel() 
    # print('Cond about',condest(pfuns@Afuns))
    w = pfun(d)
    rho = w@d
    err0 = np.sqrt(rho)
    s = w
    u = 0*d.copy()
    for it in range(maxit):
        As = Afun(s)
        alpha = rho/(As@s)
        u = u + alpha*s
        d = d - alpha*As
        w = pfun(d)
        rho1 = rho
        rho = w@d
        err = np.sqrt(rho)
        if err < tol*err0:
            break
        beta = rho/rho1
        s = w + beta*s
    if output:
        print('pcg stopped after ' + str(it) + ' iterations with relres ' + str(err/err0))
    return u,it,d

In [3]:
def refine(kvs, mult=1):
    return tuple(kv.h_refine(mult=mult) for kv in kvs)

In [33]:
u = lambda x,y: exp(x**2+y**2)
f = lambda x,y: -4*(x**2+y**2+1)*exp(x**2+y**2)
geo=geometry.unit_square()
maxiter=8
degree = (1,2,3,4)
numdofs, assembly_t, nnz, nnzq, nnzL, nnzLq, cond, cg_it = np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter)

for p in degree:
    kvs = 2*(bspline.make_knots(p,0.0,1.0,2),)
    for i in range(maxiter):
        numdofs[(p-1)*maxiter+i]=np.prod([kv.numdofs for kv in kvs])
        t=time.time()
        Ah = assemble.stiffness(kvs, geo)
        Fh = assemble.inner_products(kvs, f, f_physical=True, geo=geo).ravel()
        assembly_t[(p-1)*maxiter+i]=time.time()-t
        nnz[(p-1)*maxiter+i]=Ah.nnz#/np.prod(Ah.shape)
        nnzq[(p-1)*maxiter+i]=100*Ah.nnz/np.prod(Ah.shape)
        
        bcs = assemble.compute_dirichlet_bcs(kvs, geo, [('bottom', u), ('top', u), ('left', u), ('right', u)])
        LS = assemble.RestrictedLinearSystem(Ah, Fh, bcs)
        solver = scipy.sparse.linalg.splu(LS.A)
        nnzL[(p-1)*maxiter+i] = solver.L.nnz#/np.prod(solver.L.shape)
        nnzLq[(p-1)*maxiter+i] = 100*solver.L.nnz/np.prod(solver.L.shape)
        cond[(p-1)*maxiter+i] = condest(LS.A)
        _, it, _ = pcg(LS.A,LS.b,maxit=1000)
        cg_it[(p-1)*maxiter+i] = it
        #bcs = assemble.compute_dirichlet_bcs(kvs, geo, [('bottom',0),('right',0),('left', 0), ('top', 0)])
        if i!=maxiter-1:
            kvs = refine(kvs)
np.savetxt('Iga.txt',np.c_[numdofs, assembly_t, nnz, nnzq, nnzL, nnzLq, cond, cg_it], fmt=('%d', '%1.3f','%d','%1.3f','%d','%1.3f', '%1.3e','%d'), delimiter = " & ")

In [34]:
numdofs, assembly_t, nnz, nnzL, cond, cg_it = np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter), np.zeros(len(degree)*maxiter)

for p in degree:
    kvs = 2*(bspline.make_knots(p,0.0,1.0,2,mult=p),)
    for i in range(maxiter):
        numdofs[(p-1)*maxiter+i]=np.prod([kv.numdofs for kv in kvs])
        t=time.time()
        Ah = assemble.stiffness(kvs, geo)
        Fh = assemble.inner_products(kvs, f, f_physical=True, geo=geo).ravel()
        assembly_t[(p-1)*maxiter+i]=time.time()-t
        nnz[(p-1)*maxiter+i]=Ah.nnz#/np.prod(Ah.shape)
        nnzq[(p-1)*maxiter+i]=100*Ah.nnz/np.prod(Ah.shape)
        
        bcs = assemble.compute_dirichlet_bcs(kvs, geo, [('bottom', u), ('top', u), ('left', u), ('right', u)])
        LS = assemble.RestrictedLinearSystem(Ah, Fh, bcs)
        solver = scipy.sparse.linalg.splu(LS.A)
        nnzL[(p-1)*maxiter+i] = solver.L.nnz#/np.prod(solver.L.shape)
        nnzLq[(p-1)*maxiter+i] = 100*solver.L.nnz/np.prod(solver.L.shape)
        cond[(p-1)*maxiter+i] = condest(LS.A)
        _, it, _ = pcg(LS.A,LS.b,maxit=1000)
        cg_it[(p-1)*maxiter+i] = it
        #bcs = assemble.compute_dirichlet_bcs(kvs, geo, [('bottom',0),('right',0),('left', 0), ('top', 0)])
        if i!=maxiter-1:
            kvs = refine(kvs,mult=p)
np.savetxt('FEM.txt',np.c_[numdofs, assembly_t, nnz, nnzq, nnzL, nnzLq, cond, cg_it], fmt=('%d', '%1.3f','%d','%1.3f','%d','%1.3f', '%1.3e','%d'), delimiter = " & ")

In [4]:
Ah = assemble.stiffness(2*(bspline.make_knots(4,0.0,1.0,10,mult=1),), geo=geometry.unit_cube(2))

In [38]:
print("%1.0e" % 300.0)

3e+00


In [5]:
print(Ah.nnz)
print(Ah.nnz/(np.prod(Ah.shape)))

1191016
0.15817920254315804


In [6]:
Ah = assemble.stiffness(3*(bspline.make_knots(4,0.0,1.0,10,mult=4),), geo=geometry.unit_cube(3))

In [7]:
print(Ah.nnz)
print(Ah.nnz/(np.prod(Ah.shape)))

13997521
0.0029467818577920764


In [181]:
kvs = 2*(bspline.make_knots(2,0.0,1.0,3,mult=2),)

In [182]:
kvs[0].first_active_at(0.75)

4

In [184]:
kvs[0].numdofs

7

In [ ]:
from scipy.sparse.linalg import onenormest, splu, LinearOperator

def condest(A):
    luA = splu(A)
    iA = LinearOperator(luA.shape, matvec = lambda x : luA.solve(x), rmatvec = lambda x : luA.solve(x))
    return onenormest(iA)*onenormest(A)